In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

Using TensorFlow backend.


In [40]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [3]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

Wall time: 578 ms


In [43]:
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import os
os.listdir()

['.ipynb_checkpoints',
 'kernel56398426dd.ipynb',
 'sample_submission.csv',
 'sample_submission_usrypCc.zip',
 'test.csv',
 'test_XEV14AD.zip',
 'train.csv',
 'train_3PIRKSI.zip',
 'Untitled3.ipynb']

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
subm = pd.read_csv('sample_submission.csv')

In [7]:
train.head(5)

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [8]:
train['tag'].value_counts()

O                4446206
B-indications      53003
I-indications      44624
Name: tag, dtype: int64

In [60]:
train[train['tag'].isin(['B-indications','I-indications'])].head()

,id,Doc_ID,Sent_ID,Word,tag
171,172,1,8,strategies,B-indications
211,212,2,10,MICROCEPHALIA,B-indications
212,213,2,10,VERA,I-indications
233,234,3,12,reactive,B-indications
234,235,3,12,hyperemia,I-indications


In [10]:
train.shape
test.shape

(4543833, 5)

(2994463, 4)

In [11]:
train.nunique()

id         4543833
Doc_ID       30000
Sent_ID     191282
Word        184505
tag              3
dtype: int64

In [12]:
words = list(set(train["Word"].values))
words.append("ENDPAD")

In [13]:
n_words = len(words); n_words

184507

In [14]:
tags = list(set(train["tag"].values))

In [15]:
n_tags = len(tags); n_tags

3

In [16]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [61]:
class SentenceGetterTest(object):
    
    def __init__(self, data):
        self.n_sent = 191283
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w) for w in s["Word"].values.tolist()]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [62]:
# agg_func = lambda s: [(w, t) for w, t in zip(train["Word"].values.tolist(),train["tag"].values.tolist())]
# grouped = train[:500].groupby("Sent_ID").apply(agg_func)
# grouped[1]
# sentences = [s for s in grouped]

In [63]:
testgetter = SentenceGetterTest(test)

In [64]:
testgetter.get_next()

['CCCVA', ',', 'MANOVA', ',', 'my', 'black', 'hen', '.']

In [66]:
test_sentence = testgetter.sentences

In [18]:
getter = SentenceGetter(train)

In [19]:
sent = getter.get_next()

In [20]:
print(sent)

[('Obesity', 'O'), ('in', 'O'), ('Low-', 'O'), ('and', 'O'), ('Middle-Income', 'O'), ('Countries', 'O'), (':', 'O'), ('Burden', 'O'), (',', 'O'), ('Drivers', 'O'), (',', 'O'), ('and', 'O'), ('Emerging', 'O'), ('Challenges', 'O'), ('.', 'O')]


In [21]:
sentences = getter.sentences

In [22]:
len(sentences)

191282

In [23]:
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [24]:
word2idx["We"]

1200

In [25]:
tag2idx["O"]

0

In [26]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [27]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

In [28]:
X[1]

array([  1200, 101391,  32560, 179125,   9770, 183561,  31955,  70557,
       107790, 149100,  71840,  56592, 149100,  10293,  68664,  67573,
        10293,  90986,   4029, 149100,  70075, 107593,  70075, 133176,
       169991,  10293, 158301,  53125,  36286, 183909, 181585,   7847,
        10293, 141698,     29,  73768,  57940,  94535, 126824, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506])

In [72]:
x_test_sent  = pad_sequences(maxlen=max_len, sequences= [[word2idx.get(w, 0) for w in s] for s in test_sentence], padding="post", value=n_words - 1)

In [70]:
# x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
#                             padding="post", value=0, maxlen=max_len)

In [29]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [30]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [31]:
y[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [32]:
from keras.utils import to_categorical

In [33]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [36]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [37]:
model = Model(input, out)

In [38]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [39]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=1,
                    validation_split=0.1, verbose=1)

Train on 154937 samples, validate on 17216 samples
Epoch 1/1
154937/154937 [==============================] - 1009s 7ms/step - loss: 0.0199 - crf_viterbi_accuracy: 0.9915 - val_loss: 0.0121 - val_crf_viterbi_accuracy: 0.9942


In [46]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [42]:
test_pred = model.predict(X_te, verbose=1)

19129/19129 [==============================] - 40s 2ms/step


In [44]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [47]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 57.9%


In [48]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

indications       0.70      0.49      0.58      5201

avg / total       0.70      0.49      0.58      5201



In [49]:
i = 1927
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

Word           ||True ||Pred
disregarded    : O     O
PIP-positive   : O     O
Isoproterenol-Induced: O     O
slice-by-slice : O     O
subchannels    : O     O
beta=          : O     O
AH-6809        : O     O
-hept-2-enal   : O     O
Babyhaler      : O     O
finite-difference: O     O
PIP-positive   : O     O
plasmon-enhanced: O     O
cutánea        : O     O
anti-neuropathic-pain: O     O
hemodynamics   : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtailed      : O     O
curtai

In [50]:
test_sentence = ["type", "2", "diabetes", "Fellow", "of", "the", "Royal", "Society", ",", "a", "lifetime", "member",
                 "of", "the", "Pontifical", "Academy", "of", "Sciences", ",", "and", "a", "recipient", "of",
                 "the", "Presidential", "Medal", "of", "type", "2", "diabetes", "highest", "civilian", "award",
                 "in", "the", "United", "States", "."]

In [51]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [52]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
type           : O    
2              : O    
diabetes       : O    
Fellow         : O    
of             : O    
the            : O    
Royal          : O    
Society        : O    
,              : O    
a              : O    
lifetime       : O    
member         : O    
of             : O    
the            : O    
Pontifical     : O    
Academy        : O    
of             : O    
Sciences       : O    
,              : O    
and            : O    
a              : O    
recipient      : O    
of             : O    
the            : O    
Presidential   : O    
Medal          : O    
of             : O    
type           : B-indications
2              : I-indications
diabetes       : I-indications
highest        : O    
civilian       : O    
award          : O    
in             : O    
the            : O    
United         : O    
States         : O    
.              : O    


In [76]:
x_test_sent[0]

array([     0, 149100, 104699, 149100, 112449, 108416, 123668, 126824,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506])

In [1]:
len(x_test_sent)

NameError: name 'x_test_sent' is not defined

In [ ]:
words.append(w)
predss.append(tags[pred])

In [105]:
words = []
predss=[]
for i in range(len(x_test_sent)):
    if i%10000 ==0:
        print(i)
    idxs = i
    p = model.predict(np.array([x_test_sent[idxs]]))
    p = np.argmax(p, axis=-1)
    #print(test_sentence[idxs])
#     print(tags[p[0]])
    for w, pred in zip(test_sentence[idxs], p[0]):
        #print(w, tags[pred])
        words.append(w)
        predss.append(tags[pred])
#     break

KeyboardInterrupt: 

In [113]:
predss

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [109]:
test.shape
len(words)
len(predss)

(2994463, 4)

1276818

1276818

In [112]:
set(predss)

{'B-indications', 'I-indications', 'O'}

AttributeError: 'list' object has no attribute 'value_counts'

In [93]:
idxs = 125838
p = model.predict(np.array([x_test_sent[idxs]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
# test_sentence = 
for w, pred in zip(test_sentence[idxs], p[0]):
    print(w, tags[pred])
#     print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
< O
em O
> O
< O
strong O
> O
Bulletin O
of O
Chemical O
Reaction O
Engineering O
& O
amp O
; O
Catalysis O
< O
/strong O
> O
< O
/em O
> O
, O
8 O
( O
1 O
) O
: O
14-33 O
. O
